In [2]:
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoresummaryv2
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoremiscv2
from nba_api.stats.endpoints import boxscorefourfactorsv2
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoreusagev2
from nba_api.stats.endpoints import boxscoreplayertrackv2



In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd

In [220]:
%run -i functions.ipynb

Epoch 1/50
72/72 [==============================] - 4s 7ms/step - loss: 0.0098 - val_loss: 2.6114e-05
Epoch 2/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0069 - val_loss: 4.0494e-04
Epoch 3/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0091 - val_loss: 0.0035
Epoch 4/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0037 - val_loss: 0.0144
Epoch 5/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0057 - val_loss: 0.0227
Epoch 6/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0042 - val_loss: 0.0244
Epoch 7/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0142 - val_loss: 0.0253
Epoch 8/50
72/72 [==============================] - 0s 6ms/step - loss: 0.0038 - val_loss: 0.0254
Epoch 9/50
72/72 [==============================] - 0s 5ms/step - loss: 0.0138 - val_loss: 0.0243
Epoch 10/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0100 - val_loss: 0.0239
Epoch 11/50

In [185]:
from datetime import date
month = str(date.today().month)
day = str(date.today().day)



In [106]:
old_reg = pd.read_csv('reg_current.csv')
old_adv = pd.read_csv('adv_current.csv')
old_four = pd.read_csv('four_current.csv')
old_misc = pd.read_csv('misc_current.csv')


In [ ]:
def rem_zeros(game_id):
    new_id = game_id
    if(type(game_id) == str):
        new_id = int(game_id[2:])
    return new_id
    



In [168]:
def update_features(old_reg,old_adv,old_four,old_misc):
    last_day = old_reg['GAME_DATE'].iloc[-1]
    addition = pull_curr_reg(last_day)
    addition = addition.dropna()
    new_data = get_current_data(addition)
    addition['GAME_ID'] = addition.apply(lambda row : rem_zeros(row['GAME_ID']), axis = 1)
    new_adv = new_data[0]
    new_four = new_data[1]
    new_misc = new_data[2]
    new_adv = new_adv.dropna()
    new_four = new_four.dropna()
    new_misc = new_misc.dropna()
    new_adv['GAME_ID'] = new_adv.apply(lambda row : rem_zeros(row['GAME_ID']), axis = 1)
    new_four['GAME_ID'] = new_four.apply(lambda row : rem_zeros(row['GAME_ID']), axis = 1)
    new_misc['GAME_ID'] = new_misc.apply(lambda row : rem_zeros(row['GAME_ID']), axis = 1)

    old_reg = old_reg.append(addition)
    old_reg.to_csv('reg_current.csv')
    
    old_adv = old_adv.append(new_adv)
    old_adv.to_csv('adv_current.csv')
    
    old_four = old_four.append(new_four)
    old_four.to_csv('four_current.csv')
    
    old_misc = old_misc.append(new_misc)
    old_misc.to_csv('misc_current.csv')

    totalnew = order_features(feature_clean_current(combine_features_current(old_reg,old_misc,old_adv,old_four)))
    totalnew = totalnew.reset_index(drop=True)
    
    return totalnew

   
    #totalnew = feature_clean_current(get_current_data(addition))
    #totalnew= totalnew.reset_index(drop=True)
    #old = old.append(totalnew)
    #old.iloc[0,1] = str(date.today())
    #old.to_csv('old_features.csv')
    #return old
    #except:
     #   update_features(old)
    
    
    
    
    

In [169]:
totalnew = update_features(old_reg,old_adv,old_four,old_misc)

0022200641
0022200642
0022200643
0022200644
0022200645
0022200646
0022200647
0022200648
0022200649
0022200650
0022200651
0022200641
0022200642
0022200643
0022200644
0022200645
0022200646
0022200647
0022200648
0022200649
0022200650
0022200651
0022200641
0022200642
0022200643
0022200644
0022200645
0022200646
0022200647
0022200648
0022200649
0022200650
0022200651


In [221]:
file = 'schedule/' + month + '.csv'
schedule = pd.read_csv(file)
schedule = month_clean(schedule,old_reg)


In [222]:
schedule

,home,away,month,day
0,1610612743,1610612738,1,1
1,1610612763,1610612758,1,1
2,1610612749,1610612764,1,1
3,1610612752,1610612756,1,2
4,1610612766,1610612747,1,2
...,...,...,...,...
217,1610612739,1610612748,1,31
218,1610612752,1610612747,1,31
219,1610612741,1610612746,1,31
220,1610612749,1610612766,1,31


In [183]:

current_features = get_current_features(totalnew)


In [223]:
schedule = schedule[schedule['day'] == day]
predictions = []
for i in range(schedule.shape[0]):
    home_id = schedule.iloc[i,0]
    away_id = schedule.iloc[i,1]
    game_features = get_matchup_features(home_id,away_id,current_features)
    prediction = model.predict(game_features)
    print(predictions)
    predictions.append(prediction[0])

predictions

1/1 [==============================] - 0s 128ms/step
[]
1/1 [==============================] - 0s 27ms/step
[array([0.05940443, 0.02972724, 0.9108683 ], dtype=float32)]
1/1 [==============================] - 0s 32ms/step
[array([0.05940443, 0.02972724, 0.9108683 ], dtype=float32), array([0.05315479, 0.07944217, 0.86740303], dtype=float32)]
1/1 [==============================] - 0s 27ms/step
[array([0.05940443, 0.02972724, 0.9108683 ], dtype=float32), array([0.05315479, 0.07944217, 0.86740303], dtype=float32), array([0.01843048, 0.01327998, 0.96828955], dtype=float32)]
1/1 [==============================] - 0s 29ms/step
[array([0.05940443, 0.02972724, 0.9108683 ], dtype=float32), array([0.05315479, 0.07944217, 0.86740303], dtype=float32), array([0.01843048, 0.01327998, 0.96828955], dtype=float32), array([0.10984151, 0.21130066, 0.6788578 ], dtype=float32)]
1/1 [==============================] - 0s 30ms/step
[array([0.05940443, 0.02972724, 0.9108683 ], dtype=float32), array([0.05315479, 

[array([0.05940443, 0.02972724, 0.9108683 ], dtype=float32),
 array([0.05315479, 0.07944217, 0.86740303], dtype=float32),
 array([0.01843048, 0.01327998, 0.96828955], dtype=float32),
 array([0.10984151, 0.21130066, 0.6788578 ], dtype=float32),
 array([0.03804761, 0.03610446, 0.9258479 ], dtype=float32),
 array([0.04329792, 0.2071086 , 0.7495935 ], dtype=float32),
 array([0.10352617, 0.10804766, 0.7884261 ], dtype=float32),
 array([0.07876327, 0.21452788, 0.7067088 ], dtype=float32)]

In [224]:
schedule

,home,away,month,day
101,1610612765,1610612752,1,15
102,1610612746,1610612745,1,15
103,1610612741,1610612744,1,15
104,1610612751,1610612760,1,15
105,1610612759,1610612758,1,15
106,1610612743,1610612753,1,15
107,1610612757,1610612742,1,15
108,1610612747,1610612755,1,15


In [225]:
old_reg

,Unnamed: 0,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,home
0,8.0,22022,1610612755,PHI,Philadelphia 76ers,22200001,2022-10-18,PHI @ BOS,L,239,...,4.0,27.0,31.0,16,8.0,3,14,25,-9.0,0
1,6.0,22022,1610612738,BOS,Boston Celtics,22200001,2022-10-18,BOS vs. PHI,W,240,...,6.0,30.0,36.0,24,8.0,3,10,24,9.0,1
2,6.0,22022,1610612747,LAL,Los Angeles Lakers,22200002,2022-10-18,LAL @ GSW,L,241,...,9.0,39.0,48.0,23,12.0,4,21,18,-14.0,0
3,8.0,22022,1610612744,GSW,Golden State Warriors,22200002,2022-10-18,GSW vs. LAL,W,241,...,11.0,37.0,48.0,31,11.0,4,18,23,14.0,1
4,8.0,22022,1610612765,DET,Detroit Pistons,22200003,2022-10-19,DET vs. ORL,W,239,...,12.0,29.0,41.0,31,11.0,4,12,21,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,NaN,22022,1610612762,UTA,Utah Jazz,22200638,2023-01-13,UTA vs. ORL,W,240,...,17.0,42.0,59.0,15,9.0,11,24,18,4.0,1
1038,NaN,22022,1610612743,DEN,Denver Nuggets,22200639,2023-01-13,DEN @ LAC,None,180,...,7.0,29.0,36.0,22,4.0,3,8,9,5.0,0
1039,NaN,22022,1610612746,LAC,LA Clippers,22200639,2023-01-13,LAC vs. DEN,None,180,...,7.0,26.0,33.0,17,5.0,2,6,13,-5.0,1
1040,NaN,22022,1610612758,SAC,Sacramento Kings,22200640,2023-01-13,SAC vs. HOU,None,240,...,12.0,25.0,37.0,26,7.0,5,12,14,20.0,1


In [136]:
month

'11'

In [137]:
file = 'schedule/' + '11' + '.csv'
schedule = pd.read_csv(file)

In [138]:
schedule

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Arena,Notes
0,Tue Nov 1 2022,7:30p,Chicago Bulls,108.0,Brooklyn Nets,99.0,Box Score,NaN,17732.0,Barclays Center,NaN
1,Tue Nov 1 2022,7:30p,Golden State Warriors,109.0,Miami Heat,116.0,Box Score,NaN,19600.0,FTX Arena,NaN
2,Tue Nov 1 2022,8:00p,Orlando Magic,108.0,Oklahoma City Thunder,116.0,Box Score,NaN,13109.0,Paycom Center,NaN
3,Tue Nov 1 2022,10:00p,Minnesota Timberwolves,107.0,Phoenix Suns,116.0,Box Score,NaN,17071.0,Footprint Center,NaN
4,Wed Nov 2 2022,6:00p,Washington Wizards,121.0,Philadelphia 76ers,111.0,Box Score,NaN,19855.0,Wells Fargo Center,NaN
...,...,...,...,...,...,...,...,...,...,...,...
217,Wed Nov 30 2022,9:00p,Houston Rockets,NaN,Denver Nuggets,NaN,NaN,NaN,NaN,Ball Arena,NaN
218,Wed Nov 30 2022,9:00p,Chicago Bulls,NaN,Phoenix Suns,NaN,NaN,NaN,NaN,Footprint Center,NaN
219,Wed Nov 30 2022,9:00p,Los Angeles Clippers,NaN,Utah Jazz,NaN,NaN,NaN,NaN,Vivint Arena,NaN
220,Wed Nov 30 2022,10:00p,Indiana Pacers,NaN,Sacramento Kings,NaN,NaN,NaN,NaN,Golden 1 Center,NaN


In [211]:
i = get_team_id('Los Angeles Clippers',old_reg)

In [214]:
old_reg

,Unnamed: 0,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,home
0,8.0,22022,1610612755,PHI,Philadelphia 76ers,22200001,2022-10-18,PHI @ BOS,L,239,...,4.0,27.0,31.0,16,8.0,3,14,25,-9.0,0
1,6.0,22022,1610612738,BOS,Boston Celtics,22200001,2022-10-18,BOS vs. PHI,W,240,...,6.0,30.0,36.0,24,8.0,3,10,24,9.0,1
2,6.0,22022,1610612747,LAL,Los Angeles Lakers,22200002,2022-10-18,LAL @ GSW,L,241,...,9.0,39.0,48.0,23,12.0,4,21,18,-14.0,0
3,8.0,22022,1610612744,GSW,Golden State Warriors,22200002,2022-10-18,GSW vs. LAL,W,241,...,11.0,37.0,48.0,31,11.0,4,18,23,14.0,1
4,8.0,22022,1610612765,DET,Detroit Pistons,22200003,2022-10-19,DET vs. ORL,W,239,...,12.0,29.0,41.0,31,11.0,4,12,21,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,NaN,22022,1610612762,UTA,Utah Jazz,22200638,2023-01-13,UTA vs. ORL,W,240,...,17.0,42.0,59.0,15,9.0,11,24,18,4.0,1
1038,NaN,22022,1610612743,DEN,Denver Nuggets,22200639,2023-01-13,DEN @ LAC,None,180,...,7.0,29.0,36.0,22,4.0,3,8,9,5.0,0
1039,NaN,22022,1610612746,LAC,LA Clippers,22200639,2023-01-13,LAC vs. DEN,None,180,...,7.0,26.0,33.0,17,5.0,2,6,13,-5.0,1
1040,NaN,22022,1610612758,SAC,Sacramento Kings,22200640,2023-01-13,SAC vs. HOU,None,240,...,12.0,25.0,37.0,26,7.0,5,12,14,20.0,1


NameError: name 'pd' is not defined

In [6]:
old_features = feature_clean_current(combine_features_current(reg22,misc22,adv22,four22))

In [10]:
old_features

,TEAM_ID_A,FG_PCT_median_A,FG3_PCT_median_A,FT_PCT_median_A,PTS_per_minute_A,FGM_per_minute_A,FGA_per_minute_A,FG3M_per_minute_A,FG3A_per_minute_A,FTM_per_minute_A,...,FTA_RATE_median_A,OPP_EFG_PCT_median_A,OPP_FTA_RATE_median_A,OPP_TOV_PCT_median_A,OPP_OREB_PCT_median_A,FTA_RATE_median_B,OPP_EFG_PCT_median_B,OPP_FTA_RATE_median_B,OPP_TOV_PCT_median_B,OPP_OREB_PCT_median_B
0,1610612755,2022-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1610612738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1610612747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1610612744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1610612765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,1610612747,0.435,0.2585,0.7800,2.194444,0.822917,1.927083,0.190972,0.718750,0.357639,...,0.2415,0.505,0.2370,0.131,0.2810,0.2410,0.5135,0.2435,0.141,0.2870
234,1610612744,0.4595,0.3485,0.7650,2.416452,0.876607,1.884319,0.298201,0.861183,0.365039,...,0.2170,0.535,0.2785,0.143,0.3075,0.2825,0.5220,0.2740,0.134,0.2655
235,1610612753,0.4525,0.3075,0.7515,2.205729,0.804688,1.773438,0.203125,0.656250,0.393229,...,0.2825,0.522,0.2740,0.134,0.2655,0.2170,0.5350,0.2785,0.143,0.3075
236,1610612760,0.447,0.2940,0.8000,2.316716,0.876833,1.982405,0.217009,0.736070,0.346041,...,0.2230,0.506,0.2500,0.167,0.3000,0.2280,0.5470,0.2450,0.121,0.2750


In [8]:
reg22

,Unnamed: 0,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,home
0,8,22022,1610612755,PHI,Philadelphia 76ers,22200001,2022-10-18,PHI @ BOS,L,239,...,4.0,27.0,31.0,16,8.0,3,14,25,-9.0,0
1,6,22022,1610612738,BOS,Boston Celtics,22200001,2022-10-18,BOS vs. PHI,W,240,...,6.0,30.0,36.0,24,8.0,3,10,24,9.0,1
2,6,22022,1610612747,LAL,Los Angeles Lakers,22200002,2022-10-18,LAL @ GSW,L,241,...,9.0,39.0,48.0,23,12.0,4,21,18,-14.0,0
3,8,22022,1610612744,GSW,Golden State Warriors,22200002,2022-10-18,GSW vs. LAL,W,241,...,11.0,37.0,48.0,31,11.0,4,18,23,14.0,1
4,8,22022,1610612765,DET,Detroit Pistons,22200003,2022-10-19,DET vs. ORL,W,239,...,12.0,29.0,41.0,31,11.0,4,12,21,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,0,22022,1610612747,LAL,Los Angeles Lakers,22200117,2022-11-02,LAL vs. NOP,W,264,...,11.0,45.0,56.0,30,6.0,8,17,17,3.0,1
234,0,22022,1610612744,GSW,Golden State Warriors,22200118,2022-11-03,GSW @ ORL,L,239,...,9.0,26.0,35.0,31,5.0,3,14,32,-1.0,0
235,0,22022,1610612753,ORL,Orlando Magic,22200118,2022-11-03,ORL vs. GSW,W,240,...,11.0,35.0,46.0,20,9.0,6,19,22,1.0,1
236,0,22022,1610612760,OKC,Oklahoma City Thunder,22200119,2022-11-03,OKC vs. DEN,L,240,...,10.0,30.0,40.0,19,9.0,5,10,19,-12.0,1


In [23]:
old_features.iloc[0,1] = '2022-12-02'